<a href="https://colab.research.google.com/github/mdjamina/model_relatives_psr_fr/blob/model_v2/src/psr_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation modules

In [ ]:
!pip install datasets
!pip install -U transformers[torch]
!pip install -U accelerate

# Chargement du dataset

In [45]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset

#chargement de dataset à partir de Hugging Face
raw_datasets = load_dataset("djamina/relatives_psr")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/392 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/99 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/55 [00:00<?, ? examples/s]

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'psr_tags', 'psr_seq_tags'],
        num_rows: 392
    })
    validation: Dataset({
        features: ['id', 'tokens', 'psr_tags', 'psr_seq_tags'],
        num_rows: 99
    })
    test: Dataset({
        features: ['id', 'tokens', 'psr_tags', 'psr_seq_tags'],
        num_rows: 55
    })
})

# Préparation des données

In [18]:
task = "psr_seq" # Should be one of "psr" or "psr_seq"

model_checkpoint = "almanach/camembert-large"


In [13]:
def show(words, labels,label_names):
  line1 = ""
  line2 = ""
  for word, label in zip(words, labels):
      full_label = label_names[label]
      max_length = max(len(word), len(full_label))
      line1 += word + " " * (max_length - len(word) + 1)
      line2 += full_label + " " * (max_length - len(full_label) + 1)

  # Affichage aligné des tokens et de leurs tags
  print(line1)
  print(line2)

In [21]:
label_names = raw_datasets["train"].features[f"{task}_tags"].feature.names
print(f'label_names : {label_names}')
print("-----------------")

label_names : ['O', 'B-DET', 'I-DET', 'B-APPO', 'I-APPO', 'B-AMBIGUE', 'I-AMBIGUE']
-----------------


In [22]:

# Extraction des features 'psr_tags' à partir du dataset


words = raw_datasets["train"][205]["tokens"]
labels = raw_datasets["train"][205][f"{task}_tags"]

show(words, labels,label_names)



S' agit - il de réformer le droit du travail pour libérer , dynamiser l' emploi au bénéfice d' abord de ceux qui   n'    en    ont   pas   ? 
O  O    O O  O  O        O  O     O  O       O    O       O O         O  O      O  O        O  O     O  O    B-DET I-DET I-DET I-DET I-DET O 


## Tokenizer




In [23]:
 #Importation de la classe AutoTokenizer de la bibliothèque 'transformers'
from transformers import CamembertTokenizerFast


tokenizer = CamembertTokenizerFast.from_pretrained(model_checkpoint, force_download=True)

sentencepiece.bpe.model:   0%|          | 0.00/809k [00:00<?, ?B/s]

In [24]:
# Vérification si le tokenizer est une version rapide

tokenizer.is_fast

True

### Traitement des labels

In [25]:
# cette méthode permet d'aligner les labels avec les nouveaux tokens générés par le tokenizer
#pour aligner les labels avec les tokens
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [26]:
# cette méthode permet de tokeniser le dataset (par ligne) et d'aligner les labels avec les tokens générés par le tokenizer

def tokenize_and_align_labels(rows):
    tokenized_inputs = tokenizer(
        rows["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = rows[f"{task}_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs




### Tokenization

Application de la méthode 'tokenize_and_align_labels' sur le dataset 'raw_datasets'


pour obtenir le dataset 'tokenized_datasets' qui sera utilisé pour l'entraînement du modèle

In [27]:
# psr_tags

tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/392 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/55 [00:00<?, ? examples/s]

In [28]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 392
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 99
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 55
    })
})

In [31]:
print(tokenized_datasets["train"][15]["labels"])

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 0, 0, -100]


# Evaluate

In [32]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Méthode pour calculer les métriques d'évaluation
# On utilise les métriques de classification NER (Named Entity Recognition)
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [
        [label_names[l] for l in label if l != -100] for label in labels
    ]
    true_predictions = [
        [label_names[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Flatten the lists to calculate metrics
    true_labels_flat = [item for sublist in true_labels for item in sublist]
    true_predictions_flat = [item for sublist in true_predictions for item in sublist]

    results = {
        "precision": precision_score(true_labels_flat, true_predictions_flat, average='macro',zero_division=1),
        "recall": recall_score(true_labels_flat, true_predictions_flat, average='macro'),
        "f1": f1_score(true_labels_flat, true_predictions_flat, average='macro'),
        "accuracy": accuracy_score(true_labels_flat, true_predictions_flat),
    }
    return results

# Train

In [33]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [34]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    3,    4,    4,    4,    4,    4,    4,    4,
            4,    4,    0,    0, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    1,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    0,    0, -100]])

In [35]:
label_names = raw_datasets["train"].features[f"{task}_tags"].feature.names

id2label = {x:y for x,y in enumerate(label_names)}
label2id = {y:x for x,y in enumerate(label_names)}
label2id

{'O': 0,
 'B-DET': 1,
 'I-DET': 2,
 'B-APPO': 3,
 'I-APPO': 4,
 'B-AMBIGUE': 5,
 'I-AMBIGUE': 6}

In [36]:
from transformers import CamembertConfig, CamembertForTokenClassification

# Récupérer les noms des labels

num_labels = len(label_names)

# Définir la configuration du modèle
config = CamembertConfig(
    vocab_size=500_000,  # Taille du vocabulaire
    num_labels=num_labels,  # Nombre de labels dans votre dataset
    hidden_size=768,  # Taille des embeddings
    num_attention_heads=12,  #le lien entre les tokens, jusqu'au 12 eme token
    num_hidden_layers=12,  # Nombre de couches
)

config.label2id = label2id
config.id2label = id2label

# Initialiser le modèle avec la configuration
model = CamembertForTokenClassification(config)

In [37]:
label_names

['O', 'B-DET', 'I-DET', 'B-APPO', 'I-APPO', 'B-AMBIGUE', 'I-AMBIGUE']

In [38]:
from transformers import TrainingArguments


model_name =  f"relatives_{task}"

args = TrainingArguments(
    model_name,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    #weight_decay=0.01,
    push_to_hub=False,

)

In [39]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


In [40]:
history = trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.728474,0.712516,0.328366,0.286215,0.761129
2,No log,0.678902,0.639746,0.339962,0.309833,0.768507
3,No log,0.673896,0.524727,0.375362,0.368484,0.767235
4,No log,0.665041,0.459995,0.398413,0.388556,0.755533
5,No log,0.664189,0.505244,0.408228,0.405057,0.766980


In [46]:
trainer.push_to_hub(commit_message="Training complete")

events.out.tfevents.1717609183.85b7333225dc.3235.0:   0%|          | 0.00/7.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

sentencepiece.bpe.model:   0%|          | 0.00/809k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/djamina/relatives_psr_seq/commit/e40c139cced1d69a8a44364ec94ac40e3a258891', commit_message='Training complete', commit_description='', oid='e40c139cced1d69a8a44364ec94ac40e3a258891', pr_url=None, pr_revision=None, pr_num=None)

## Inference